# Vector Store

## Produce text on dogs and cats

In [11]:
# Load simple sentences
_texts = [
    "Dogs are great companions, "
    "known for their loyalty and friendliness.",
    "Cats are independent pets that often enjoy their own space.",
]

## Load PDF

In [12]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = "./Rabbits and Guinea Pigs.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

## Setup vector store and retriever tool

In [13]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import create_retriever_tool

_embeddings = OpenAIEmbeddings()
vector_store = InMemoryVectorStore(_embeddings)

vector_store.add_texts(_texts)
vector_store.add_documents(all_splits)

document_tool = create_retriever_tool(
    vector_store.as_retriever(),
    "pet_information_retriever",
    "Fetches information about pets (dogs, cats, rabbits, and guinea pigs)."
)

## Create agent

In [14]:
from langchain.agents import create_agent
from langchain_ollama.chat_models import ChatOllama
from langchain_openai import ChatOpenAI

ollama_model = ChatOllama(model="mistral:latest")
openai_model = ChatOpenAI(model="gpt-3.5-turbo")

use_ollama = True
selected_model = ollama_model if use_ollama else openai_model

agent = create_agent(
    selected_model,
    tools=[document_tool],
    prompt="Act as an assistant. Call the document tool that you have available to answer questions.",
)

In [15]:
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.messages.chat import ChatMessage

result = agent.invoke((
    {"messages": [HumanMessage("What are rabbits known for?")]}
))

In [16]:
import pprint as pp

for m in result['messages']:
    m.pretty_print()

================================ Human Message =================================

What are rabbits known for?
================================== Ai Message ==================================

 [{"name":"pet_information_retriever","arguments":{"query":"Rabbits"}}]

According to the information retrieved, Rabbits are known for their long ears, soft fur, and twitching noses. They are herbivores that consume a variety of plant materials. Rabbits are also famous for their high reproductive rate.


## For multiple documents

In [17]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="Doc 2",
    metadata={"source": "tweet"},
)

In [18]:
documents = [document_1, document_2]

vector_store.add_documents(documents=documents)

['c3366212-6783-4c71-b29a-18631129a47d',
 'b3da40a5-0dd9-4e8b-97c2-01c128853784']